In [ ]:
!pip install dw
!pip install skforecast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carga Fichero
import pandas as pd
datos = pd.read_csv('Modelar_UH2022.txt',sep='|',parse_dates=['SAMPLETIME'],infer_datetime_format=True)

In [ ]:
#Modelar_UH2022_10_Contadores.txt
datos[datos['ID']<11].to_csv('Modelar_UH2022_10_Contadores.txt',sep='|',index=False,float_format='%.0f')

In [ ]:
import pandas as pd
from tqdm import tqdm
from datetime import date
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.preprocessing import StandardScaler
import math
import numpy as np


class Contador(object):
    def __init__(self, datos_contador, ID,lags,max_depth,estimators ):
        self.datos_contador = datos_contador
        self.ID = ID
        self.lags=lags
        self.max_depth=max_depth
        self.estimators=estimators
        self.Preprocesado()
        self.SinDatos = False  # NO TIENE DATOS DATASET VACIO
        self.outlier = False
        self.dias = 0
        self.RMSE_Train = 0
        self.RMSE_Test = 0
        self.datos_test = pd.DataFrame()
        # {'ID','Dia_1':0,'Dia_2':0,'Dia_3':0,'Dia_4':0,'Dia_5':0,'Dia_6':0','Dia_7':0,'Semana_1':0,'Semana_2':0}
        self.Predicciones = [self.ID,0,0,0,0,0,0,0,0,0]
        
        if not datos_contador.empty:
            # CALCULO LECTURA CONTADOR Y CONSUMO JUNTANDO DECIMALES
            #self.datos_contador['CONSUMO'] = self.datos_contador['DELTAINTEGER'] + self.__Convertir_Entero(self.datos_contador['DELTATHOUSANDTH'])
            #self.datos_contador['LECTURA'] = self.datos_contador['READINGINTEGER'] + self.__Convertir_Entero(self.datos_contador['READINGTHOUSANDTH'])
            self.datos_contador['CONSUMO'] = self.datos_contador['DELTAINTEGER'] + (self.datos_contador['DELTATHOUSANDTH'] / 100)
            self.datos_contador['LECTURA'] = self.datos_contador['READINGINTEGER'] + (self.datos_contador['READINGTHOUSANDTH'] / 100)
            
            # CONSUMO POR DIAS
            aux = self.datos_contador.groupby([self.datos_contador['SAMPLETIME'].dt.year,self.datos_contador['SAMPLETIME'].dt.month, self.datos_contador['SAMPLETIME'].dt.day]).agg({'CONSUMO':sum})
            if len(aux.index)>1:    # SI TENEMOS MAS DE UN DIA DE DATOS
                self.datos_consumo = pd.DataFrame(data={'Fecha':self.__Convertir_fecha(aux),'Consumo':aux.squeeze().values})
            else:   # SI TENEMOS SOLO UN DIA DE DATOS
                self.datos_consumo = pd.DataFrame(data={'Fecha':self.__Convertir_fecha(aux),'Consumo':datos_contador['CONSUMO'].sum()})
            del aux
            
            # ESTADISTICAS
            self.dias = len(self.datos_consumo.index)
            self.datos_consumo['Cuenta'] = self.datos_contador.groupby([self.datos_contador['SAMPLETIME'].dt.year,self.datos_contador['SAMPLETIME'].dt.month, self.datos_contador['SAMPLETIME'].dt.day]).agg({'CONSUMO':'count'}).values
            self.fecha_inicio_contador= self.datos_consumo['Fecha'][0]
            self.fecha_fin_contador=self.datos_consumo['Fecha'].tail(1)
        else:
            self.datos_consumo = pd.DataFrame()
            self.SinDatos = True
            self.fecha_fin_contador = None

        # GENERACION DEL MODELO
        self.modelo()
    
    def __Convertir_fecha(self, serie):
        fechas = []
        for año, mes, dia in serie.index:
            fechas.append(date(año,mes,dia))
        return fechas

    def __Convertir_Entero(self,serie):
        resultado = {}
        indices = []
        
        for indice, numero in serie.items():
            if numero < 10:
                resultado[indice] = numero/10
                indices.append(indice)
            elif numero < 100:
                resultado[indice] = numero/100
                indices.append(indice)
            elif numero >= 100:
                print('ERROR parte decimal mayor de 100')
        return pd.Series(data=resultado, index=indices)
    
    def Preprocesado(self):
        # READINGINTEGER	READINGTHOUSANDTH	DELTAINTEGER	DELTATHOUSANDTH
        # VALORES  NULOS 
        # VALORES ENTEROS INTERPOLAMOS
        # JOSEP

        self.datos_contador['READINGINTEGER'].interpolate(inplace=True)
        self.datos_contador['DELTAINTEGER'].interpolate(inplace=True)
        
        # VALORES DECIMALES SUSTITUIMOS POR CERO
        self.datos_contador['READINGTHOUSANDTH'].fillna(0,inplace=True)
        self.datos_contador['DELTATHOUSANDTH'].fillna(0,inplace=True)
        
        # ELIMINAR VALORES NEGATIVOS
        indices = self.datos_contador[self.datos_contador['READINGINTEGER'] < 0].index
        self.datos_contador.drop(indices,inplace=True)
        indices = self.datos_contador[self.datos_contador['DELTAINTEGER'] < 0].index
        self.datos_contador.drop(indices,inplace=True)

        # RESAMPLE
        
        # CONDICION ULTIMOS DIAS VER GRAFICA Y FIJAR NUMERO
        
        # CONTADORES CON POCOS DATOS VEMOS SI SON NUEVOS TIENEN DATOS EN LOS ULTIMOS DIAS HACEMOS MEDIA
        # EN EL CASO DE NO TENER LOS ULTIMOS DATOS PONEMOS CERO

    #def PreModelo(self):
        # PREPROCESADO PARA INTRODUCIR LOS DATOS EN EL MODELO
        # FORMATO DATOS EN FUNCIÓN DEL MODELO
        # PACO

    def modelo(self,umbrales=180):
        # DECISION DEL MODELO A TOMAR EN FUNCION DE LA ESTADISTICAS
        # CARGA DEL MODELO
        # PACO
        
        if self.dias > umbrales:
           self.model = ForecasterAutoreg(regressor=RandomForestRegressor(random_state=123,n_estimators=self.estimators,max_depth=self.max_depth), lags=self.lags)
        else:
         #   if self.dias > 0 :
            if self.dias > 0: 
                if (self.fecha_fin_contador > date(2020,1,17)).bool() :    # HACE MEDIA SI TENEMOS DATOS EN LOS ULTIMOS 14 DIAS
                    self.model = self.datos_consumo['Consumo'].mean()
                else:
                    self.model = 0
            else:
                self.model = 0
    

class CajaMar_Water(object):
    def __init__(self,Ruta_Datos,Fecha_Inicio=date(2019,2,1),Fecha_Fin=date(2020,1,31)):
        self.Ruta_Datos = Ruta_Datos
        self.Fecha_Inicio = Fecha_Inicio
        self.Lags_list=[4,2]      ###############DATOS GRID
        self.estimators_list=[100,500]   ##################DATOS GRID
        self.max_depth_list=[3]     #################DATOS GRID
        self.Fecha_Fin = Fecha_Fin
        self.datos = pd.DataFrame()
        self.contadores = []
        self.dias_test = 14     # DIAS QUE SE RESERVAN PARA TEST EMPEZANDO POR LA FECHA MAS RECIENTE
        self.Cargar_Datos()
        self.RMSE_Test_Promedio=0
        self.grid(self.Lags_list,self.estimators_list,self.max_depth_list)
    

    def Cargar_Datos(self):
        print('CARGANDO DATOS.....')
        self.datos = pd.read_csv(self.Ruta_Datos,sep='|',parse_dates=['SAMPLETIME'],infer_datetime_format=True,encoding='utf-8')
        self.datos.sort_values(by=['ID','SAMPLETIME'],inplace=True)

        # CONSUMO POR DIAS Y CONTADORES
        #print('AGRUPANDO CONSUMOS POR DIAS.....')
        #self.datos_consumo = self.datos.groupby([self.datos['ID'],self.datos['SAMPLETIME'].dt.year,self.datos['SAMPLETIME'].dt.month, self.datos['SAMPLETIME'].dt.day]).agg({'CONSUMO':sum})
        

        print('PROCESO DE CARGA FINALIZADO')
    
    def PartirDatos(self):
        # Separar datos de entrenamiento y test / validacion (dias=0 no hace validacion)
        if self.dias_test > 0:     
            for contador in tqdm(self.contadores):
                if contador.SinDatos == False:      # COMPROBAMOS QUE HAY ALGUN REGISTRO
                    if (contador.fecha_fin_contador==self.Fecha_Fin).bool():
                        if contador.dias > self.dias_test + 1:    # COMPROBAMOS QUE HAY SUFICIENTES DATOS PARA HACER PARTICION
                            contador.datos_test = contador.datos_consumo.tail(self.dias_test)
                            contador.datos_consumo = contador.datos_consumo[0:-self.dias_test]

    def grid(self,Lags_list,estimators_list, max_depth_list):
      self.RMSEs=[]
      self.grid_values=[]
      for lags in self.Lags_list:
        for estimators in self.estimators_list:
              for max_depth in self.max_depth_list:
                self.lags=lags
                self.max_depth=max_depth
                self.estimators=estimators
                # GENERANDO CONTADORES
                print('START:')
                for ID in tqdm(pd.unique(self.datos['ID'])):
                  # ALMACENAMOS ID DE CONTADOR Y BORRAMOS LA COLUMNA
                  self.contadores.append(Contador(self.datos[self.datos['ID']==ID].drop('ID',axis=1),ID,lags,max_depth,estimators))
                self.PartirDatos()
                print("Datos Cargados para: "+"lags= "+str(lags)+",max_depth= "+str(max_depth)+",max_estimators= "+str(estimators))
                self.Entrenamiento()
                self.Calculo_RMSE()
                self.RMSEs.append(self.RMSE_Test_Promedio)
                self.grid_values.append([lags,estimators, max_depth])
      self.min_RMSEs=min(self.RMSEs)
      print('\033[1m'+"RMSE más bajo: "+str(self.min_RMSEs))
      best_values_index=self.RMSEs.index(self.min_RMSEs)
      print('\033[1m'+"Best score: "+"lags= "+str(self.grid_values[best_values_index][0])+",estimators= "+str(self.grid_values[best_values_index][1])+",max_depth= "+str(self.grid_values[best_values_index][2]))





    def Entrenamiento(self):
        # Entrenamiento de los modelos en self.contadores(lista) 
        # iterar lista y llamar a self.contadores[iter].modelo.train
        # Rellena la variable self.contadores.Predicciones
        
        for contador in tqdm(self.contadores):
            if isinstance(contador.model,ForecasterAutoreg):    # COMPROBAMOS QUE ES UN MODELO
                contador.model.fit(y=contador.datos_consumo['Consumo'])
                predict = contador.model.predict(steps=self.dias_test)
                index = 1
                for predict_dia in predict[0:7]:    # GUARDA 7 DIAS PREDICCION
                    contador.Predicciones[index] = predict_dia
                    index += 1
                contador.Predicciones[8]=predict[0:7].sum()     # GUARDA SEMANA1 PREDICCION
                contador.Predicciones[9]=predict[7:14].sum()    # GUARDA SEMANA2 PREDICCION
            else:
              for index in range(1,8):
                    contador.Predicciones[index] = contador.model  # GUARDAMOS MEDIA EN LOS 7 DIAS
              contador.Predicciones[8]=contador.model*7     # GUARDAMOS MEDIA SEMANA1
              contador.Predicciones[9]=contador.model*7     # GUARDAMOS MEDIA SEMANA2
                  
                    
              
        

    def Calculo_RMSE(self):
        # Calculo rmse utlizando datos de test
        RMSE_Todos = []
        for contador in tqdm(self.contadores):
            #if contador.dias > 180 and len(contador.datos_test) == self.dias_test:  # VER LIMITE NO DEPENDE DE LA CLASE CONTADOR
            if len(contador.datos_test) == self.dias_test:
                media_RMSE_semanal=math.sqrt(mean_squared_error([contador.datos_test['Consumo'][0:7].sum(), contador.datos_test['Consumo'][7:14].sum()] , [contador.Predicciones[8], contador.Predicciones[9]]))
                contador.RMSE_Test = 0.5*math.sqrt(mean_squared_error(contador.datos_test['Consumo'][0:7],contador.Predicciones[1:8]))+0.5*media_RMSE_semanal
                RMSE_Todos.append(contador.RMSE_Test)
        self.RMSE_Test_Promedio = sum(RMSE_Todos) / len(RMSE_Todos)
        print(self.RMSE_Test_Promedio)
    
    def GenerarResultados(self,equipo='Team2021IA3',numeral=1):
        # Mediante los datos predicciones generamos txt con el formato del datathon
        # Separando campos con “|”, el valor de la predicción en litros, y los decimales con “.” 2 decimales
        # Fichero: "Cajamar_Universitat de València (UV)_Team2021IA3_numeral.txt"
        
        columnas = ['ID','Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7','Semana_1','Semana_2']
        datos = []

        for contador in tqdm(self.contadores):
            datos.append(contador.Predicciones)
        
       
        dataset = pd.DataFrame(datos,columns=columnas)
        dataset.to_csv('Cajamar_Universitat de València (UV)_' + equipo + '_' + str(numeral) + '.txt',header=False ,sep='|' ,index=False ,decimal='.' ,float_format='%.2f')
         
    def GraficasEstadisticas(self):
        # GRÁFICAS MAS REPRESENTATIVAS DE LOS CONTADORES
        ID=[]
        dias=[]
        for contador in tqdm(self.contadores):
            ID.append(contador.ID)
            dias.append(contador.dias)

        plt.plot(ID,dias)

In [ ]:
#Proyecto = CajaMar_Water(path+'Modelar_UH2022_10_Contadores.txt')
Proyecto = CajaMar_Water('Modelar_UH2022.txt')

CARGANDO DATOS.....
PROCESO DE CARGA FINALIZADO
START:


100%|██████████| 11/11 [00:00<00:00, 1973.20it/s]


Datos Cargados para: lags= 4,max_depth= 3,max_estimators= 100


100%|██████████| 11/11 [00:00<00:00, 826.85it/s]


255.50955368550873
START:


...

202.21552955596874
START:


100%|██████████| 44/44 [00:00<00:00, 2921.66it/s]


Datos Cargados para: lags= 2,max_depth= 3,max_estimators= 500


100%|██████████| 44/44 [00:00<00:00, 988.69it/s]

206.2513510574054
RMSE más bajo: 202.21552955596874
Best score: lags= 2,estimators= 100,max_depth= 3
